## Identifying and resolving conflicts in a batch of VI files

Andes Reinspection 
BGS 
Data information can be found at https://data.desi.lbl.gov/desi/spectro/prospect/target-reinspection-andes/

/global/cfs/cdirs/desi/spectro/redux/andes


448 visual inspections of a total of 150 unique objects

32 conflicts

In [412]:
save_filename='truth_table_Andes_reinspection_BGS_70500_20200303_3'

In [413]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

import VI_utils as VI
sys.path.append("/global/homes/r/tlan/prospect/prospect/py")

from prospect import utils_specviewer,plotframes
#import desispec
# library location will change ..
sys.path.append("/global/u2/t/tlan/python/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [414]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/'
tiles = ['70500']
nights = ['20200303']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
petals = ['0','3','6','7']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
del obs_db['70500-20200228']
obs_db.keys()

dict_keys(['70500-20200303'])

In [415]:
pd.set_option('display.max_rows', 10)

In [416]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/projects/VI_files/Andes_reinspection/BGS/'

In [417]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(7,11):
    pattern = "desi*_"+str(70500)+"*"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

print(vi_files)

desi*_70500*7_*.csv
desi*_70500*8_*.csv
desi*_70500*9_*.csv
desi*_70500*10_*.csv
['desi-vi_BGS_reinspection_70500_7_SYB.csv', 'desi-vi_BGS_reinspection_70500_7_AMA.csv', 'desi-vi_BGS_reinspection_70500_7_CHH.csv', 'desi-vi_BGS_reinspection_70500_8_AMA.csv', 'desi-vi_BGS_reinspection_70500_8_CHH.csv', 'desi-vi_BGS_reinspection_70500_8_SYB.csv', 'desi-vi_BGS_reinspection_70500_9_SYB.csv', 'desi-vi_BGS_reinspection_70500_9_AMA.csv', 'desi-vi_BGS_reinspection_70500_9_CHH.csv', 'desi-vi_BGS_reinspection_70500_10_SMC.csv', 'desi-vi_BGS_reinspection_70500_10_AMA.csv', 'desi-vi_BGS_reinspection_70500_10_CHH.csv']


In [418]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = ",", engine='python')
print(vi_files[0])
for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = ",", engine='python')
    vi = vi.append(vi2, ignore_index=True)
#print(vi)    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TARGETID'])

print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_BGS_reinspection_70500_7_SYB.csv
desi-vi_BGS_reinspection_70500_7_AMA.csv
desi-vi_BGS_reinspection_70500_7_CHH.csv
desi-vi_BGS_reinspection_70500_8_AMA.csv
desi-vi_BGS_reinspection_70500_8_CHH.csv
desi-vi_BGS_reinspection_70500_8_SYB.csv
desi-vi_BGS_reinspection_70500_9_SYB.csv
desi-vi_BGS_reinspection_70500_9_AMA.csv
desi-vi_BGS_reinspection_70500_9_CHH.csv
desi-vi_BGS_reinspection_70500_10_SMC.csv
desi-vi_BGS_reinspection_70500_10_AMA.csv
desi-vi_BGS_reinspection_70500_10_CHH.csv
There are 598 visual inspections of a total of 200 unique objects
That gives 2.99 inspections per object.


In [419]:
#vi is a dataframe
#display(vi)

In [420]:
#vi.keys()
len(vi)

598

# Merge with zbest files
#Add: fiberID, delta_chi2, flux information,.. anything else?


In [421]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')

for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

#tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_panda()    
EXPID = list(set(tf['EXPID']))[0]
tf = tf[tf['EXPID']==EXPID]
tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()
print(len(tspec_df))
print(len(tf_df))
#tf_df = tf_df.rename(columns={"TargetID": "TARGETID"})
#tspec_df = tspec_df.rename(columns={"TargetID": "TARGETID"})
#print(len(vi))
vi = vi.merge(tf_df, how='left', on='TARGETID')
vi = vi.merge(tspec_df, how='left', on='TARGETID')
print(len(vi))

2000
2000
598


In [422]:
print(vi.keys())
#print(len(vi))
#vi['best redshift']=vi['VI_z']

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')


### Adding a bunch of useful columns

In [423]:
#vi['VI_z']
import VI_utils as VI
#vi['VI_z']
#bad_str = [s is None for s in vi['VI_comment']]
#vi['VI_comment'].isnull()
#bad_str = [s is None for s in vi['VI_comment']]
#print(bad_str)
#vi.loc[vi['VI_comment'].isnull(),'VI_comment']='--'
#vi['VI_comment']
#vi['all VI comments'] = vi.groupby('TARGETID')['VI_comment'].transform(lambda x: '|'.join(x))
#print(vi['all VI comments'])
vi = VI.prep_new_columns(vi)
#vi['best redshift'] = vi['VI_z']
#test = vi['VI_z']
#vi['best redshift'].isnull()
#vi.loc[vi['best redshift'].isnull(), 'best redshift'] = vi.loc[vi['best redshift'].isnull(), 'Redrock_z']
#vi['best redshift']

In [424]:
#check all the new columns (keys) have been added correctly
display(vi)
#print(vi.keys())

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35191228169454444,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4472,SYB,...,521.691587,0.4472,GALAXY,2.666667,1,0.000000,0.4472,H; K abs|weak continuum/features|weak absorptions,3,none
1,35191228169454659,52932,20200303,70500,0.34.4,-1,-1,QSO,1.2139,SYB,...,51.914930,0.0885,GALAXY,2.333333,3,1.033900,1.2139,Can hand-fit Ha; SII; OII lines.|weak features...,3,none
2,35191228169455210,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.0582,SYB,...,16177.217726,0.0582,GALAXY,4.000000,0,0.000000,0.0582,Ha; NII; OII; Ca abs|--|--,3,none
3,35191228169455339,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2203,SYB,...,2883.672994,0.2203,GALAXY,3.666667,1,0.000000,0.2203,H; K; Mg abs|--|--,3,none
4,35191228169455568,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.5496,SYB,...,57.920409,0.5496,GALAXY,2.000000,2,0.000000,0.5496,K; Balmer abs|weak features/continuum|--,3,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,35191243474469278,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2000,CHH,...,880.877408,0.2000,GALAXY,3.666667,1,0.000000,0.2000,--|--|--,3,none
594,35191243474469310,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3336,CHH,...,3200.682983,0.3336,GALAXY,4.000000,0,0.000000,0.3336,--|--|--,3,none
595,35191243474470161,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.0742,CHH,...,71.539206,1.0742,GALAXY,1.000000,2,0.000000,1.0742,--|weak features and continuum|--,3,none
596,35191235886972941,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.6708,CHH,...,30.597106,0.6708,GALAXY,2.333333,3,0.634034,0.6708,--|--|--,3,none


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [425]:
vi_gp = vi.groupby(['TARGETID'])
vi_conflict = VI.find_conflicts(vi_gp)

In [426]:
len(vi_gp)

200

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [427]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35191228169454659 35191228169455568 35191228173648571 35191228173649628
 35191228173649991 35191228177843458 35191228177844275 35191228194619902
 35191228194620670 35191228194622595 35191228203008416 35191228203009684
 35191228203011017 35191228203011308 35191228203011878 35191228207203531
 35191228207203773 35191228211396983 35191228211397264 35191228211397323
 35191228211398185 35191228211399171 35191228211399613 35191232015630690
 35191232019825420 35191232044993460 35191232049186476 35191232049187192
 35191232053381935 35191232053382496 35191232057576825 35191232061767736
 35191235874390207 35191235874390797 35191235874391652 35191235874393030
 35191235878585750 35191235878587725 35191235882780025 35191235886972941
 35191239682818245 35191239682818588 35191239682819126 35191239682819256
 35191239682819376 35191239682819437 35191239682820184 35191239682821246
 35191239682821445 35191239687012856 35191239687013396 35191239687014075
 35191239691206731 35

In [428]:
unique_targets[0]

35191228169454659

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [429]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TARGETID==unique_targets[conflict_id]][['TARGETID', 
       'Redrock_spectype', 'Redrock_z', 'VI_scanner', 'VI_quality', 'VI_issue',
       'VI_z', 'VI_spectype', 'VI_comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
    # VI interface in notebook
    if show_spectra:
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [430]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [431]:
show_spectra = False

In [432]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
1,35191228169454659,QSO,1.2139,SYB,4,R,0.0885,GALAXY,Can hand-fit Ha; SII; OII lines.,0.0885,GALAXY,2.333333,3,1.2139,1.0339,Can hand-fit Ha; SII; OII lines.|weak features...,3,none
50,35191228169454659,QSO,1.2139,AMA,2,NaN,NaN,NaN,weak features/continuum,1.2139,QSO,2.333333,3,1.2139,1.0339,Can hand-fit Ha; SII; OII lines.|weak features...,3,none
99,35191228169454659,QSO,1.2139,CHH,1,NaN,NaN,NaN,--,1.2139,QSO,2.333333,3,1.2139,1.0339,Can hand-fit Ha; SII; OII lines.|weak features...,3,none


In [433]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0885
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. redshift is secure. Ha and SII and OII'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
1,35191228169454659,52932,20200303,70500,0.34.4,-1,-1,QSO,1.2139,SYB,...,51.91493,0.0885,GALAXY,2.333333,3,1.0339,1.2139,Can hand-fit Ha; SII; OII lines.|weak features...,3,disagreement in quality. redshift is secure. H...
50,35191228169454659,52932,20200303,70500,0.34.4,-1,-1,QSO,1.2139,AMA,...,51.91493,0.0885,GALAXY,2.333333,3,1.0339,1.2139,Can hand-fit Ha; SII; OII lines.|weak features...,3,disagreement in quality. redshift is secure. H...
99,35191228169454659,52932,20200303,70500,0.34.4,-1,-1,QSO,1.2139,CHH,...,51.91493,0.0885,GALAXY,2.333333,3,1.0339,1.2139,Can hand-fit Ha; SII; OII lines.|weak features...,3,disagreement in quality. redshift is secure. H...


1     4
50    4
99    4
Name: VI_quality, dtype: int64
conflict ID =  0


In [434]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
4,35191228169455568,GALAXY,0.5496,SYB,3,NaN,NaN,NaN,K; Balmer abs,0.5496,GALAXY,2.0,2,0.5496,0.0,K; Balmer abs|weak features/continuum|--,3,none
53,35191228169455568,GALAXY,0.5496,AMA,2,NaN,NaN,NaN,weak features/continuum,0.5496,GALAXY,2.0,2,0.5496,0.0,K; Balmer abs|weak features/continuum|--,3,none
102,35191228169455568,GALAXY,0.5496,CHH,1,NaN,NaN,NaN,--,0.5496,GALAXY,2.0,2,0.5496,0.0,K; Balmer abs|weak features/continuum|--,3,none


In [435]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] =2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Quality disagreement.  Some weak absorption features'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)



,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
4,35191228169455568,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.5496,SYB,...,57.920409,0.5496,GALAXY,2.0,2,0.0,0.5496,K; Balmer abs|weak features/continuum|--,3,Quality disagreement. Some weak absorption fe...
53,35191228169455568,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.5496,AMA,...,57.920409,0.5496,GALAXY,2.0,2,0.0,0.5496,K; Balmer abs|weak features/continuum|--,3,Quality disagreement. Some weak absorption fe...
102,35191228169455568,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.5496,CHH,...,57.920409,0.5496,GALAXY,2.0,2,0.0,0.5496,K; Balmer abs|weak features/continuum|--,3,Quality disagreement. Some weak absorption fe...


4      2
53     2
102    2
Name: VI_quality, dtype: int64
conflict ID =  1


In [436]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
11,35191228173648571,QSO,1.2222,SYB,1,NaN,NaN,NaN,--,1.2222,QSO,1.666667,2,1.2222,0.074929,--|--|--,3,none
60,35191228173648571,QSO,1.2222,AMA,3,R,1.0673,NaN,--,1.0673,QSO,1.666667,2,1.2222,0.074929,--|--|--,3,none
109,35191228173648571,QSO,1.2222,CHH,1,NaN,NaN,NaN,--,1.2222,QSO,1.666667,2,1.2222,0.074929,--|--|--,3,none


In [437]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2320
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. z is not secure. Bad calibration'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
11,35191228173648571,52932,20200303,70500,0.34.4,-1,-1,QSO,1.2222,SYB,...,173.215296,1.2222,QSO,1.666667,2,0.074929,1.2222,--|--|--,3,disagreement in quality. z is not secure. Bad ...
60,35191228173648571,52932,20200303,70500,0.34.4,-1,-1,QSO,1.2222,AMA,...,173.215296,1.0673,QSO,1.666667,2,0.074929,1.2222,--|--|--,3,disagreement in quality. z is not secure. Bad ...
109,35191228173648571,52932,20200303,70500,0.34.4,-1,-1,QSO,1.2222,CHH,...,173.215296,1.2222,QSO,1.666667,2,0.074929,1.2222,--|--|--,3,disagreement in quality. z is not secure. Bad ...


conflict ID =  2


In [438]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=True)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
17,35191228173649628,GALAXY,0.3997,SYB,1,NaN,NaN,NaN,--,0.3997,GALAXY,2.666667,3,0.3997,0.0,--|--|--,3,none
66,35191228173649628,GALAXY,0.3997,AMA,3,NaN,NaN,NaN,--,0.3997,GALAXY,2.666667,3,0.3997,0.0,--|--|--,3,none
115,35191228173649628,GALAXY,0.3997,CHH,4,NaN,NaN,NaN,--,0.3997,GALAXY,2.666667,3,0.3997,0.0,--|--|--,3,none


Loading BokehJS ...

In [439]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5128
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY' 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in quality. bad calibration. redshift is secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
17,35191228173649628,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3997,SYB,...,355.378629,0.3997,GALAXY,2.666667,3,0.0,0.3997,--|--|--,3,disagreement in quality. bad calibration. reds...
66,35191228173649628,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3997,AMA,...,355.378629,0.3997,GALAXY,2.666667,3,0.0,0.3997,--|--|--,3,disagreement in quality. bad calibration. reds...
115,35191228173649628,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3997,CHH,...,355.378629,0.3997,GALAXY,2.666667,3,0.0,0.3997,--|--|--,3,disagreement in quality. bad calibration. reds...


17     4
66     4
115    4
Name: VI_quality, dtype: int64
conflict ID =  3


In [440]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
67,35191228173649991,GALAXY,0.1841,AMA,-1,C,1.0845,QSO,possible QSO object,1.0845,QSO,1.0,4,0.1841,0.760409,possible QSO object|--,2,none
116,35191228173649991,GALAXY,0.1841,CHH,3,NaN,NaN,NaN,--,0.1841,GALAXY,1.0,4,0.1841,0.760409,possible QSO object|--,2,none


In [441]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3210
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Ha SII look good'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print(vi[vi.TARGETID==unique_targets[conflict_id]]['VI_quality'])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
67,35191228173649991,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.1841,AMA,...,2605.849124,1.0845,QSO,1.0,4,0.760409,0.1841,possible QSO object|--,2,disagreement in flag. Ha SII look good
116,35191228173649991,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.1841,CHH,...,2605.849124,0.1841,GALAXY,1.0,4,0.760409,0.1841,possible QSO object|--,2,disagreement in flag. Ha SII look good


67     4
116    4
Name: VI_quality, dtype: int64
conflict ID =  4


In [442]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
20,35191228177843458,GALAXY,0.4244,SYB,1,NaN,NaN,NaN,--,0.4244,GALAXY,2.0,2,0.4244,0.0,--|weak features/continuum|multiple absorption...,3,none
70,35191228177843458,GALAXY,0.4244,AMA,2,NaN,NaN,NaN,weak features/continuum,0.4244,GALAXY,2.0,2,0.4244,0.0,--|weak features/continuum|multiple absorption...,3,none
119,35191228177843458,GALAXY,0.4244,CHH,3,NaN,NaN,NaN,multiple absorption but bad continuum,0.4244,GALAXY,2.0,2,0.4244,0.0,--|weak features/continuum|multiple absorption...,3,none


In [443]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Bad calibration. Redshift is secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
20,35191228177843458,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4244,SYB,...,470.898218,0.4244,GALAXY,2.0,2,0.0,0.4244,--|weak features/continuum|multiple absorption...,3,disagreement in flag. Bad calibration. Redshif...
70,35191228177843458,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4244,AMA,...,470.898218,0.4244,GALAXY,2.0,2,0.0,0.4244,--|weak features/continuum|multiple absorption...,3,disagreement in flag. Bad calibration. Redshif...
119,35191228177843458,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4244,CHH,...,470.898218,0.4244,GALAXY,2.0,2,0.0,0.4244,--|weak features/continuum|multiple absorption...,3,disagreement in flag. Bad calibration. Redshif...


conflict ID =  5


In [444]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
22,35191228177844275,GALAXY,0.3139,SYB,3,NaN,NaN,NaN,--,0.3139,GALAXY,3.0,2,0.3139,0.0,--|weak features; okay continuum|--,3,none
72,35191228177844275,GALAXY,0.3139,AMA,2,NaN,NaN,NaN,weak features; okay continuum,0.3139,GALAXY,3.0,2,0.3139,0.0,--|weak features; okay continuum|--,3,none
121,35191228177844275,GALAXY,0.3139,CHH,4,NaN,NaN,NaN,--,0.3139,GALAXY,3.0,2,0.3139,0.0,--|weak features; okay continuum|--,3,none


In [445]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 1.5
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift is secure.'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
22,35191228177844275,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3139,SYB,...,72.446822,0.3139,GALAXY,3.0,2,0.0,0.3139,--|weak features; okay continuum|--,3,Redshift is secure.
72,35191228177844275,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3139,AMA,...,72.446822,0.3139,GALAXY,3.0,2,0.0,0.3139,--|weak features; okay continuum|--,3,Redshift is secure.
121,35191228177844275,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3139,CHH,...,72.446822,0.3139,GALAXY,3.0,2,0.0,0.3139,--|weak features; okay continuum|--,3,Redshift is secure.


conflict ID =  6


In [446]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
27,35191228194619902,QSO,1.1417,SYB,1,NaN,NaN,NaN,--,1.1417,QSO,2.0,3,1.1417,0.047593,--|--|--,3,none
77,35191228194619902,QSO,1.1417,AMA,4,R,1.0444,NaN,--,1.0444,QSO,2.0,3,1.1417,0.047593,--|--|--,3,none
126,35191228194619902,QSO,1.1417,CHH,1,NaN,NaN,NaN,--,1.1417,QSO,2.0,3,1.1417,0.047593,--|--|--,3,none


In [447]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1847
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Calibration drives the best fit  Redshift is not secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
27,35191228194619902,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1417,SYB,...,126.143234,1.1417,QSO,2.0,3,0.047593,1.1417,--|--|--,3,disagreement in flag. Calibration drives the b...
77,35191228194619902,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1417,AMA,...,126.143234,1.0444,QSO,2.0,3,0.047593,1.1417,--|--|--,3,disagreement in flag. Calibration drives the b...
126,35191228194619902,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1417,CHH,...,126.143234,1.1417,QSO,2.0,3,0.047593,1.1417,--|--|--,3,disagreement in flag. Calibration drives the b...


conflict ID =  7


In [448]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
31,35191228194620670,QSO,1.1438,SYB,1,NaN,NaN,NaN,--,1.1438,QSO,2.0,3,1.1438,0.0,--|--|--,3,none
80,35191228194620670,QSO,1.1438,AMA,4,NaN,NaN,NaN,--,1.1438,QSO,2.0,3,1.1438,0.0,--|--|--,3,none
130,35191228194620670,QSO,1.1438,CHH,1,NaN,NaN,NaN,--,1.1438,QSO,2.0,3,1.1438,0.0,--|--|--,3,none


In [449]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1920
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift is secure. Calibration drvies the best fit.'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
31,35191228194620670,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1438,SYB,...,47.919155,0.192,QSO,2.0,3,0.0,1.1438,--|--|--,3,Redshift is secure. Calibration drvies the bes...
80,35191228194620670,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1438,AMA,...,47.919155,0.192,QSO,2.0,3,0.0,1.1438,--|--|--,3,Redshift is secure. Calibration drvies the bes...
130,35191228194620670,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1438,CHH,...,47.919155,0.192,QSO,2.0,3,0.0,1.1438,--|--|--,3,Redshift is secure. Calibration drvies the bes...


conflict ID =  8


In [450]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
34,35191228194622595,GALAXY,0.4847,SYB,1,NaN,NaN,NaN,--,0.4847,GALAXY,1.666667,2,0.4847,0.0,--|--|--,3,none
83,35191228194622595,GALAXY,0.4847,AMA,3,NaN,NaN,NaN,--,0.4847,GALAXY,1.666667,2,0.4847,0.0,--|--|--,3,none
133,35191228194622595,GALAXY,0.4847,CHH,1,NaN,NaN,NaN,--,0.4847,GALAXY,1.666667,2,0.4847,0.0,--|--|--,3,none


In [451]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1327
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Redshift is secure. Bad calibration'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
34,35191228194622595,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4847,SYB,...,542.381956,0.1327,GALAXY,1.666667,2,0.0,0.4847,--|--|--,3,disagreement in flag. Redshift is secure. Bad ...
83,35191228194622595,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4847,AMA,...,542.381956,0.1327,GALAXY,1.666667,2,0.0,0.4847,--|--|--,3,disagreement in flag. Redshift is secure. Bad ...
133,35191228194622595,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4847,CHH,...,542.381956,0.1327,GALAXY,1.666667,2,0.0,0.4847,--|--|--,3,disagreement in flag. Redshift is secure. Bad ...


conflict ID =  9


In [452]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
47,35191228203008416,GALAXY,0.7945,SYB,1,NaN,NaN,NaN,--,0.7945,GALAXY,1.666667,2,0.7945,0.0,--|--|--,3,none
96,35191228203008416,GALAXY,0.7945,AMA,3,NaN,NaN,NaN,--,0.7945,GALAXY,1.666667,2,0.7945,0.0,--|--|--,3,none
146,35191228203008416,GALAXY,0.7945,CHH,1,NaN,NaN,NaN,--,0.7945,GALAXY,1.666667,2,0.7945,0.0,--|--|--,3,none


In [453]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2780
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. Possible Ha at 0.2780'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
47,35191228203008416,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.7945,SYB,...,35.040787,0.278,GALAXY,1.666667,2,0.0,0.7945,--|--|--,3,flag disagreement. Possible Ha at 0.2780
96,35191228203008416,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.7945,AMA,...,35.040787,0.278,GALAXY,1.666667,2,0.0,0.7945,--|--|--,3,flag disagreement. Possible Ha at 0.2780
146,35191228203008416,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.7945,CHH,...,35.040787,0.278,GALAXY,1.666667,2,0.0,0.7945,--|--|--,3,flag disagreement. Possible Ha at 0.2780


conflict ID =  10


In [454]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
170,35191228203009684,QSO,2.0978,AMA,3,NaN,NaN,NaN,--,2.0978,QSO,1.666667,2,2.0978,0.0,--|--|Bump in spectrum on top of B-R splice,3,none
220,35191228203009684,QSO,2.0978,CHH,1,NaN,NaN,NaN,--,2.0978,QSO,1.666667,2,2.0978,0.0,--|--|Bump in spectrum on top of B-R splice,3,none
270,35191228203009684,QSO,2.0978,SYB,1,S,NaN,NaN,Bump in spectrum on top of B-R splice,2.0978,QSO,1.666667,2,2.0978,0.0,--|--|Bump in spectrum on top of B-R splice,3,none


In [455]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0662
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Strange feature driving the best fit.'
#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
170,35191228203009684,52932,20200303,70500,0.34.4,-1,-1,QSO,2.0978,AMA,...,126.66751,2.0978,QSO,1.666667,2,0.0,2.0978,--|--|Bump in spectrum on top of B-R splice,3,Strange feature driving the best fit.
220,35191228203009684,52932,20200303,70500,0.34.4,-1,-1,QSO,2.0978,CHH,...,126.66751,2.0978,QSO,1.666667,2,0.0,2.0978,--|--|Bump in spectrum on top of B-R splice,3,Strange feature driving the best fit.
270,35191228203009684,52932,20200303,70500,0.34.4,-1,-1,QSO,2.0978,SYB,...,126.66751,2.0978,QSO,1.666667,2,0.0,2.0978,--|--|Bump in spectrum on top of B-R splice,3,Strange feature driving the best fit.


conflict ID =  11


In [456]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
171,35191228203011017,GALAXY,1.2139,AMA,4,NaN,NaN,NaN,--,1.2139,GALAXY,2.666667,2,1.2139,0.0,--|--|OII emission line; no doublet?,3,none
221,35191228203011017,GALAXY,1.2139,CHH,2,NaN,NaN,NaN,--,1.2139,GALAXY,2.666667,2,1.2139,0.0,--|--|OII emission line; no doublet?,3,none
271,35191228203011017,GALAXY,1.2139,SYB,2,NaN,NaN,NaN,OII emission line; no doublet?,1.2139,GALAXY,2.666667,2,1.2139,0.0,--|--|OII emission line; no doublet?,3,none


In [457]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0810
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Possible [OII]. Could be sky line'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
171,35191228203011017,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.2139,AMA,...,161.737728,1.2139,GALAXY,2.666667,2,0.0,1.2139,--|--|OII emission line; no doublet?,3,Possible [OII]. Could be sky line
221,35191228203011017,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.2139,CHH,...,161.737728,1.2139,GALAXY,2.666667,2,0.0,1.2139,--|--|OII emission line; no doublet?,3,Possible [OII]. Could be sky line
271,35191228203011017,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.2139,SYB,...,161.737728,1.2139,GALAXY,2.666667,2,0.0,1.2139,--|--|OII emission line; no doublet?,3,Possible [OII]. Could be sky line


conflict ID =  12


In [458]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
173,35191228203011308,GALAXY,0.3326,AMA,2,NaN,NaN,NaN,no identifiable features,0.3326,GALAXY,3.0,2,0.3326,0.0,no identifiable features|multiple weak absorpt...,3,none
223,35191228203011308,GALAXY,0.3326,CHH,3,NaN,NaN,NaN,multiple weak absorption,0.3326,GALAXY,3.0,2,0.3326,0.0,no identifiable features|multiple weak absorpt...,3,none
273,35191228203011308,GALAXY,0.3326,SYB,4,NaN,NaN,NaN,Balmer abs lines,0.3326,GALAXY,3.0,2,0.3326,0.0,no identifiable features|multiple weak absorpt...,3,none


In [459]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Absorption features are secure. Calibration '

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
173,35191228203011308,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3326,AMA,...,380.342998,0.3326,GALAXY,3.0,2,0.0,0.3326,no identifiable features|multiple weak absorpt...,3,disagreement in flag. Absorption features are ...
223,35191228203011308,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3326,CHH,...,380.342998,0.3326,GALAXY,3.0,2,0.0,0.3326,no identifiable features|multiple weak absorpt...,3,disagreement in flag. Absorption features are ...
273,35191228203011308,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3326,SYB,...,380.342998,0.3326,GALAXY,3.0,2,0.0,0.3326,no identifiable features|multiple weak absorpt...,3,disagreement in flag. Absorption features are ...


conflict ID =  13


In [460]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
174,35191228203011878,GALAXY,0.2576,AMA,3,NaN,NaN,NaN,--,0.2576,GALAXY,2.333333,2,0.2576,0.0,--|--|Ha; SII emit; K; H abs,3,none
224,35191228203011878,GALAXY,0.2576,CHH,1,NaN,NaN,NaN,--,0.2576,GALAXY,2.333333,2,0.2576,0.0,--|--|Ha; SII emit; K; H abs,3,none
274,35191228203011878,GALAXY,0.2576,SYB,3,NaN,NaN,NaN,Ha; SII emit; K; H abs,0.2576,GALAXY,2.333333,2,0.2576,0.0,--|--|Ha; SII emit; K; H abs,3,none


In [461]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Redshift is secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
174,35191228203011878,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2576,AMA,...,442.439107,0.2576,GALAXY,2.333333,2,0.0,0.2576,--|--|Ha; SII emit; K; H abs,3,Redshift is secure
224,35191228203011878,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2576,CHH,...,442.439107,0.2576,GALAXY,2.333333,2,0.0,0.2576,--|--|Ha; SII emit; K; H abs,3,Redshift is secure
274,35191228203011878,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2576,SYB,...,442.439107,0.2576,GALAXY,2.333333,2,0.0,0.2576,--|--|Ha; SII emit; K; H abs,3,Redshift is secure


conflict ID =  14


In [462]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
175,35191228207203531,GALAXY,0.492,AMA,4,NaN,NaN,NaN,--,0.492,GALAXY,2.0,3,0.492,0.0,--|--|No features; jump in B-R splice,3,none
225,35191228207203531,GALAXY,0.492,CHH,1,NaN,NaN,NaN,--,0.492,GALAXY,2.0,3,0.492,0.0,--|--|No features; jump in B-R splice,3,none
275,35191228207203531,GALAXY,0.492,SYB,1,S,NaN,NaN,No features; jump in B-R splice,0.492,GALAXY,2.0,3,0.492,0.0,--|--|No features; jump in B-R splice,3,none


In [463]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1955
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration | redshift secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
175,35191228207203531,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.492,AMA,...,3.125207,0.1955,GALAXY,2.0,3,0.0,0.492,--|--|No features; jump in B-R splice,3,Bad calibration | redshift secure
225,35191228207203531,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.492,CHH,...,3.125207,0.1955,GALAXY,2.0,3,0.0,0.492,--|--|No features; jump in B-R splice,3,Bad calibration | redshift secure
275,35191228207203531,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.492,SYB,...,3.125207,0.1955,GALAXY,2.0,3,0.0,0.492,--|--|No features; jump in B-R splice,3,Bad calibration | redshift secure


conflict ID =  15


In [464]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
178,35191228207203773,QSO,1.1561,AMA,3,NaN,NaN,NaN,--,1.1561,QSO,1.666667,2,1.1561,0.0,--|--|Poor residuals; B-R normalization mismatch,3,none
228,35191228207203773,QSO,1.1561,CHH,1,NaN,NaN,NaN,--,1.1561,QSO,1.666667,2,1.1561,0.0,--|--|Poor residuals; B-R normalization mismatch,3,none
278,35191228207203773,QSO,1.1561,SYB,1,S,NaN,NaN,Poor residuals; B-R normalization mismatch,1.1561,QSO,1.666667,2,1.1561,0.0,--|--|Poor residuals; B-R normalization mismatch,3,none


In [465]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.2409
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, z is not secure. Ha SII possible at 0.1995'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
178,35191228207203773,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1561,AMA,...,113.460908,1.1561,QSO,1.666667,2,0.0,1.1561,--|--|Poor residuals; B-R normalization mismatch,3,"disagreement in flag, z is not secure. Ha SII ..."
228,35191228207203773,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1561,CHH,...,113.460908,1.1561,QSO,1.666667,2,0.0,1.1561,--|--|Poor residuals; B-R normalization mismatch,3,"disagreement in flag, z is not secure. Ha SII ..."
278,35191228207203773,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1561,SYB,...,113.460908,1.1561,QSO,1.666667,2,0.0,1.1561,--|--|Poor residuals; B-R normalization mismatch,3,"disagreement in flag, z is not secure. Ha SII ..."


conflict ID =  16


In [466]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
184,35191228211396983,GALAXY,0.3755,AMA,-1,RC,1.0775,NaN,possible QSO,1.0775,GALAXY,2.333333,5,0.3755,0.51036,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,none
234,35191228211396983,GALAXY,0.3755,CHH,4,NaN,NaN,NaN,--,0.3755,GALAXY,2.333333,5,0.3755,0.51036,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,none
284,35191228211396983,GALAXY,0.3755,SYB,4,NaN,NaN,NaN,Ha + abs lines; B-R-Z splice issues?,0.3755,GALAXY,2.333333,5,0.3755,0.51036,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,none


In [467]:
#copy this text to a new cell to resolve the conflict

vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration.'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
184,35191228211396983,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3755,AMA,...,2927.864409,1.0775,GALAXY,2.333333,5,0.51036,0.3755,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,Bad calibration.
234,35191228211396983,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3755,CHH,...,2927.864409,0.3755,GALAXY,2.333333,5,0.51036,0.3755,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,Bad calibration.
284,35191228211396983,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3755,SYB,...,2927.864409,0.3755,GALAXY,2.333333,5,0.51036,0.3755,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,Bad calibration.


conflict ID =  17


In [468]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
185,35191228211397264,QSO,1.1576,AMA,3,NaN,NaN,NaN,--,1.1576,QSO,1.666667,2,1.1576,0.0,--|--|No clear lines; B-R-Z splice issues?,3,none
235,35191228211397264,QSO,1.1576,CHH,1,NaN,NaN,NaN,--,1.1576,QSO,1.666667,2,1.1576,0.0,--|--|No clear lines; B-R-Z splice issues?,3,none
285,35191228211397264,QSO,1.1576,SYB,1,NaN,NaN,NaN,No clear lines; B-R-Z splice issues?,1.1576,QSO,1.666667,2,1.1576,0.0,--|--|No clear lines; B-R-Z splice issues?,3,none


In [469]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0948
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. Redshift is not secure. Bad Calibration'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
185,35191228211397264,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1576,AMA,...,93.455038,1.1576,QSO,1.666667,2,0.0,1.1576,--|--|No clear lines; B-R-Z splice issues?,3,disagreement in flag. Redshift is not secure. ...
235,35191228211397264,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1576,CHH,...,93.455038,1.1576,QSO,1.666667,2,0.0,1.1576,--|--|No clear lines; B-R-Z splice issues?,3,disagreement in flag. Redshift is not secure. ...
285,35191228211397264,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1576,SYB,...,93.455038,1.1576,QSO,1.666667,2,0.0,1.1576,--|--|No clear lines; B-R-Z splice issues?,3,disagreement in flag. Redshift is not secure. ...


conflict ID =  18


In [470]:
#copy this text to a new cell to display the conflict
conflict_id=19
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
186,35191228211397323,GALAXY,0.2954,AMA,3,NaN,NaN,NaN,--,0.2954,GALAXY,1.666667,2,0.2954,0.0,--|--|No clear lines; B-R-Z splice issues?,3,none
236,35191228211397323,GALAXY,0.2954,CHH,1,NaN,NaN,NaN,--,0.2954,GALAXY,1.666667,2,0.2954,0.0,--|--|No clear lines; B-R-Z splice issues?,3,none
286,35191228211397323,GALAXY,0.2954,SYB,1,NaN,NaN,NaN,No clear lines; B-R-Z splice issues?,0.2954,GALAXY,1.666667,2,0.2954,0.0,--|--|No clear lines; B-R-Z splice issues?,3,none


In [471]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1940
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration.  absorption features are secure.'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
186,35191228211397323,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2954,AMA,...,90.610391,0.194,GALAXY,1.666667,2,0.0,0.2954,--|--|No clear lines; B-R-Z splice issues?,3,Bad calibration. absorption features are secure.
236,35191228211397323,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2954,CHH,...,90.610391,0.194,GALAXY,1.666667,2,0.0,0.2954,--|--|No clear lines; B-R-Z splice issues?,3,Bad calibration. absorption features are secure.
286,35191228211397323,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2954,SYB,...,90.610391,0.194,GALAXY,1.666667,2,0.0,0.2954,--|--|No clear lines; B-R-Z splice issues?,3,Bad calibration. absorption features are secure.


conflict ID =  19


In [472]:
#copy this text to a new cell to display the conflict
conflict_id=20
display_conflict(conflict_id, show_spectra=show_spectra)

,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
187,35191228211398185,QSO,1.1579,AMA,4,NaN,NaN,NaN,--,1.1579,QSO,3.0,3,1.1579,0.510183,--|--|Ha + NII + OII from 3rd fit,3,none
237,35191228211398185,QSO,1.1579,CHH,1,NaN,NaN,NaN,--,1.1579,QSO,3.0,3,1.1579,0.510183,--|--|Ha + NII + OII from 3rd fit,3,none
287,35191228211398185,QSO,1.1579,SYB,4,R,0.4289,NaN,Ha + NII + OII from 3rd fit,0.4289,QSO,3.0,3,1.1579,0.510183,--|--|Ha + NII + OII from 3rd fit,3,none


In [473]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.4289
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration.  Emission features are secure.'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
187,35191228211398185,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1579,AMA,...,75.099372,0.4289,GALAXY,3.0,3,0.510183,1.1579,--|--|Ha + NII + OII from 3rd fit,3,Bad calibration. Emission features are secure.
237,35191228211398185,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1579,CHH,...,75.099372,0.4289,GALAXY,3.0,3,0.510183,1.1579,--|--|Ha + NII + OII from 3rd fit,3,Bad calibration. Emission features are secure.
287,35191228211398185,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1579,SYB,...,75.099372,0.4289,GALAXY,3.0,3,0.510183,1.1579,--|--|Ha + NII + OII from 3rd fit,3,Bad calibration. Emission features are secure.


conflict ID =  20


In [474]:
#copy this text to a new cell to display the conflict
conflict_id=21
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
190,35191228211399171,QSO,1.2304,AMA,3,NaN,NaN,NaN,--,1.2304,QSO,1.666667,2,1.2304,0.0,--|--|B-R jumps; some skyline residuals,3,none
240,35191228211399171,QSO,1.2304,CHH,1,NaN,NaN,NaN,--,1.2304,QSO,1.666667,2,1.2304,0.0,--|--|B-R jumps; some skyline residuals,3,none
290,35191228211399171,QSO,1.2304,SYB,1,S,NaN,NaN,B-R jumps; some skyline residuals,1.2304,QSO,1.666667,2,1.2304,0.0,--|--|B-R jumps; some skyline residuals,3,none


In [475]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.3525
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement.  Bad calibration'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
190,35191228211399171,52932,20200303,70500,0.34.4,-1,-1,QSO,1.2304,AMA,...,156.23614,1.2304,QSO,1.666667,2,0.0,1.2304,--|--|B-R jumps; some skyline residuals,3,flag disagreement. Bad calibration
240,35191228211399171,52932,20200303,70500,0.34.4,-1,-1,QSO,1.2304,CHH,...,156.23614,1.2304,QSO,1.666667,2,0.0,1.2304,--|--|B-R jumps; some skyline residuals,3,flag disagreement. Bad calibration
290,35191228211399171,52932,20200303,70500,0.34.4,-1,-1,QSO,1.2304,SYB,...,156.23614,1.2304,QSO,1.666667,2,0.0,1.2304,--|--|B-R jumps; some skyline residuals,3,flag disagreement. Bad calibration


conflict ID =  21


In [476]:
#copy this text to a new cell to display the conflict
conflict_id=22
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
191,35191228211399613,QSO,1.1745,AMA,3,NaN,NaN,NaN,bad model fit,1.1745,QSO,1.666667,2,1.1745,0.0,bad model fit|--|--,3,none
241,35191228211399613,QSO,1.1745,CHH,1,NaN,NaN,NaN,--,1.1745,QSO,1.666667,2,1.1745,0.0,bad model fit|--|--,3,none
291,35191228211399613,QSO,1.1745,SYB,1,NaN,NaN,NaN,--,1.1745,QSO,1.666667,2,1.1745,0.0,bad model fit|--|--,3,none


In [477]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1395
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Redshift is secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
191,35191228211399613,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1745,AMA,...,82.757454,0.1395,GALAXY,1.666667,2,0.0,1.1745,bad model fit|--|--,3,Bad calibration. Redshift is secure
241,35191228211399613,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1745,CHH,...,82.757454,0.1395,GALAXY,1.666667,2,0.0,1.1745,bad model fit|--|--,3,Bad calibration. Redshift is secure
291,35191228211399613,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1745,SYB,...,82.757454,0.1395,GALAXY,1.666667,2,0.0,1.1745,bad model fit|--|--,3,Bad calibration. Redshift is secure


conflict ID =  22


In [478]:
#copy this text to a new cell to display the conflict

conflict_id=23
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
150,35191232015630690,GALAXY,0.6397,AMA,2,NaN,NaN,NaN,weak features,0.6397,GALAXY,3.333333,2,0.6397,0.0,weak features|--|Strong O; Hbeta,3,none
200,35191232015630690,GALAXY,0.6397,CHH,4,NaN,NaN,NaN,--,0.6397,GALAXY,3.333333,2,0.6397,0.0,weak features|--|Strong O; Hbeta,3,none
250,35191232015630690,GALAXY,0.6397,SYB,4,NaN,NaN,NaN,Strong O; Hbeta,0.6397,GALAXY,3.333333,2,0.6397,0.0,weak features|--|Strong O; Hbeta,3,none


In [479]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2370
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. OII OIII secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
150,35191232015630690,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.6397,AMA,...,35.835308,0.6397,GALAXY,3.333333,2,0.0,0.6397,weak features|--|Strong O; Hbeta,3,Bad calibration. OII OIII secure
200,35191232015630690,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.6397,CHH,...,35.835308,0.6397,GALAXY,3.333333,2,0.0,0.6397,weak features|--|Strong O; Hbeta,3,Bad calibration. OII OIII secure
250,35191232015630690,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.6397,SYB,...,35.835308,0.6397,GALAXY,3.333333,2,0.0,0.6397,weak features|--|Strong O; Hbeta,3,Bad calibration. OII OIII secure


conflict ID =  23


In [480]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
153,35191232019825420,GALAXY,1.6366,AMA,3,NaN,NaN,NaN,--,1.6366,GALAXY,2.666667,3,1.6366,1.18352,--|--|Strong abs and emission using 3rd fit,3,none
203,35191232019825420,GALAXY,1.6366,CHH,1,NaN,NaN,NaN,--,1.6366,GALAXY,2.666667,3,1.6366,1.18352,--|--|Strong abs and emission using 3rd fit,3,none
253,35191232019825420,GALAXY,1.6366,SYB,4,R,0.2075,NaN,Strong abs and emission using 3rd fit,0.2075,GALAXY,2.666667,3,1.6366,1.18352,--|--|Strong abs and emission using 3rd fit,3,none


In [481]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2075
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Ha SII NII OIII OII secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
153,35191232019825420,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.6366,AMA,...,275.582756,0.2075,GALAXY,2.666667,3,1.18352,1.6366,--|--|Strong abs and emission using 3rd fit,3,Bad calibration. Ha SII NII OIII OII secure
203,35191232019825420,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.6366,CHH,...,275.582756,0.2075,GALAXY,2.666667,3,1.18352,1.6366,--|--|Strong abs and emission using 3rd fit,3,Bad calibration. Ha SII NII OIII OII secure
253,35191232019825420,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.6366,SYB,...,275.582756,0.2075,GALAXY,2.666667,3,1.18352,1.6366,--|--|Strong abs and emission using 3rd fit,3,Bad calibration. Ha SII NII OIII OII secure


conflict ID =  24


In [482]:
#copy this text to a new cell to display the conflict
conflict_id=25
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
165,35191232044993460,GALAXY,0.2858,AMA,3,NaN,NaN,NaN,--,0.2858,GALAXY,1.666667,2,0.2858,0.0,--|--|No good lines; 4000A break in wrong loca...,3,none
215,35191232044993460,GALAXY,0.2858,CHH,1,NaN,NaN,NaN,--,0.2858,GALAXY,1.666667,2,0.2858,0.0,--|--|No good lines; 4000A break in wrong loca...,3,none
265,35191232044993460,GALAXY,0.2858,SYB,1,NaN,NaN,NaN,No good lines; 4000A break in wrong location f...,0.2858,GALAXY,1.666667,2,0.2858,0.0,--|--|No good lines; 4000A break in wrong loca...,3,none


In [483]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2263
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. No obvious features'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
165,35191232044993460,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2858,AMA,...,142.112353,0.2858,GALAXY,1.666667,2,0.0,0.2858,--|--|No good lines; 4000A break in wrong loca...,3,Bad calibration. No obvious features
215,35191232044993460,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2858,CHH,...,142.112353,0.2858,GALAXY,1.666667,2,0.0,0.2858,--|--|No good lines; 4000A break in wrong loca...,3,Bad calibration. No obvious features
265,35191232044993460,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2858,SYB,...,142.112353,0.2858,GALAXY,1.666667,2,0.0,0.2858,--|--|No good lines; 4000A break in wrong loca...,3,Bad calibration. No obvious features


conflict ID =  25


In [484]:
#copy this text to a new cell to display the conflict
conflict_id=26
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
303,35191232049186476,GALAXY,0.2675,SYB,2,NaN,NaN,NaN,Weak; maybe 4000A break,0.2675,GALAXY,2.666667,2,0.2675,0.0,Weak; maybe 4000A break|--|--,3,none
353,35191232049186476,GALAXY,0.2675,AMA,4,NaN,NaN,NaN,--,0.2675,GALAXY,2.666667,2,0.2675,0.0,Weak; maybe 4000A break|--|--,3,none
403,35191232049186476,GALAXY,0.2675,CHH,2,NaN,NaN,NaN,--,0.2675,GALAXY,2.666667,2,0.2675,0.0,Weak; maybe 4000A break|--|--,3,none


In [485]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2699
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. absorption  features are secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
303,35191232049186476,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2675,SYB,...,194.622055,0.2675,GALAXY,2.666667,2,0.0,0.2675,Weak; maybe 4000A break|--|--,3,Bad calibration. absorption features are secure
353,35191232049186476,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2675,AMA,...,194.622055,0.2675,GALAXY,2.666667,2,0.0,0.2675,Weak; maybe 4000A break|--|--,3,Bad calibration. absorption features are secure
403,35191232049186476,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2675,CHH,...,194.622055,0.2675,GALAXY,2.666667,2,0.0,0.2675,Weak; maybe 4000A break|--|--,3,Bad calibration. absorption features are secure


conflict ID =  26


In [486]:
#copy this text to a new cell to display the conflict
conflict_id=27
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
306,35191232049187192,GALAXY,0.4673,SYB,1,NaN,NaN,NaN,Maybe Ha line; poor residuals,0.4673,GALAXY,1.666667,2,0.4673,0.0,Maybe Ha line; poor residuals|--|--,3,none
356,35191232049187192,GALAXY,0.4673,AMA,3,NaN,NaN,NaN,--,0.4673,GALAXY,1.666667,2,0.4673,0.0,Maybe Ha line; poor residuals|--|--,3,none
406,35191232049187192,GALAXY,0.4673,CHH,1,NaN,NaN,NaN,--,0.4673,GALAXY,1.666667,2,0.4673,0.0,Maybe Ha line; poor residuals|--|--,3,none


In [487]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2699
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Redshift is secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
306,35191232049187192,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4673,SYB,...,411.771724,0.4673,GALAXY,1.666667,2,0.0,0.4673,Maybe Ha line; poor residuals|--|--,3,Bad calibration. Redshift is secure
356,35191232049187192,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4673,AMA,...,411.771724,0.4673,GALAXY,1.666667,2,0.0,0.4673,Maybe Ha line; poor residuals|--|--,3,Bad calibration. Redshift is secure
406,35191232049187192,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4673,CHH,...,411.771724,0.4673,GALAXY,1.666667,2,0.0,0.4673,Maybe Ha line; poor residuals|--|--,3,Bad calibration. Redshift is secure


conflict ID =  27


In [488]:
#copy this text to a new cell to display the conflict
conflict_id=28
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
315,35191232053381935,GALAXY,1.649,SYB,1,NaN,NaN,NaN,No obvious lines; plausible fit to 4000A break,1.649,GALAXY,1.666667,2,1.649,0.0,No obvious lines; plausible fit to 4000A break...,3,none
365,35191232053381935,GALAXY,1.649,AMA,3,NaN,NaN,NaN,--,1.649,GALAXY,1.666667,2,1.649,0.0,No obvious lines; plausible fit to 4000A break...,3,none
415,35191232053381935,GALAXY,1.649,CHH,1,NaN,NaN,NaN,--,1.649,GALAXY,1.666667,2,1.649,0.0,No obvious lines; plausible fit to 4000A break...,3,none


In [489]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2770
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. absorption features are robust'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
315,35191232053381935,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.649,SYB,...,93.550564,0.277,GALAXY,1.666667,2,0.0,1.649,No obvious lines; plausible fit to 4000A break...,3,Bad calibration. absorption features are robust
365,35191232053381935,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.649,AMA,...,93.550564,0.277,GALAXY,1.666667,2,0.0,1.649,No obvious lines; plausible fit to 4000A break...,3,Bad calibration. absorption features are robust
415,35191232053381935,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.649,CHH,...,93.550564,0.277,GALAXY,1.666667,2,0.0,1.649,No obvious lines; plausible fit to 4000A break...,3,Bad calibration. absorption features are robust


conflict ID =  28


In [490]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
317,35191232053382496,GALAXY,0.4144,SYB,0,S,NaN,NaN,Poor residuals in B-R and R-Z overlaps,0.4144,GALAXY,1.333333,3,0.4144,0.0,Poor residuals in B-R and R-Z overlaps|--|--,3,none
367,35191232053382496,GALAXY,0.4144,AMA,3,NaN,NaN,NaN,--,0.4144,GALAXY,1.333333,3,0.4144,0.0,Poor residuals in B-R and R-Z overlaps|--|--,3,none
417,35191232053382496,GALAXY,0.4144,CHH,1,NaN,NaN,NaN,--,0.4144,GALAXY,1.333333,3,0.4144,0.0,Poor residuals in B-R and R-Z overlaps|--|--,3,none


In [491]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2699
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. absorption features are not secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
317,35191232053382496,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4144,SYB,...,113.277323,0.4144,GALAXY,1.333333,3,0.0,0.4144,Poor residuals in B-R and R-Z overlaps|--|--,3,Bad calibration. absorption features are not s...
367,35191232053382496,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4144,AMA,...,113.277323,0.4144,GALAXY,1.333333,3,0.0,0.4144,Poor residuals in B-R and R-Z overlaps|--|--,3,Bad calibration. absorption features are not s...
417,35191232053382496,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4144,CHH,...,113.277323,0.4144,GALAXY,1.333333,3,0.0,0.4144,Poor residuals in B-R and R-Z overlaps|--|--,3,Bad calibration. absorption features are not s...


conflict ID =  29


In [492]:
#copy this text to a new cell to display the conflict
conflict_id=30
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
342,35191232057576825,GALAXY,0.2512,SYB,4,NaN,NaN,NaN,Bright Ha; OII; dip in continuum at B-R,0.2512,GALAXY,3.333333,2,0.2512,0.0,Bright Ha; OII; dip in continuum at B-R|weak H...,3,none
392,35191232057576825,GALAXY,0.2512,AMA,2,NaN,NaN,NaN,weak H alpha emission,0.2512,GALAXY,3.333333,2,0.2512,0.0,Bright Ha; OII; dip in continuum at B-R|weak H...,3,none
442,35191232057576825,GALAXY,0.2512,CHH,4,NaN,NaN,NaN,--,0.2512,GALAXY,3.333333,2,0.2512,0.0,Bright Ha; OII; dip in continuum at B-R|weak H...,3,none


In [493]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2699
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. emission features are secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
342,35191232057576825,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2512,SYB,...,724.809422,0.2512,GALAXY,3.333333,2,0.0,0.2512,Bright Ha; OII; dip in continuum at B-R|weak H...,3,Bad calibration. emission features are secure
392,35191232057576825,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2512,AMA,...,724.809422,0.2512,GALAXY,3.333333,2,0.0,0.2512,Bright Ha; OII; dip in continuum at B-R|weak H...,3,Bad calibration. emission features are secure
442,35191232057576825,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2512,CHH,...,724.809422,0.2512,GALAXY,3.333333,2,0.0,0.2512,Bright Ha; OII; dip in continuum at B-R|weak H...,3,Bad calibration. emission features are secure


conflict ID =  30


In [494]:
#copy this text to a new cell to display the conflict
conflict_id=31
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
343,35191232061767736,GALAXY,1.349,SYB,-1,R,0.3337,NaN,OII id by pipeline could be Ha (3rd fit). Ugly...,0.3337,GALAXY,1.333333,4,1.349,0.761266,OII id by pipeline could be Ha (3rd fit). Ugly...,3,none
393,35191232061767736,GALAXY,1.349,AMA,3,NaN,NaN,NaN,--,1.3490,GALAXY,1.333333,4,1.349,0.761266,OII id by pipeline could be Ha (3rd fit). Ugly...,3,none
443,35191232061767736,GALAXY,1.349,CHH,2,NaN,NaN,NaN,possibly OII emission line,1.3490,GALAXY,1.333333,4,1.349,0.761266,OII id by pipeline could be Ha (3rd fit). Ugly...,3,none


In [495]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.3337
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. emission features are secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
343,35191232061767736,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.349,SYB,...,241.123424,0.3337,GALAXY,1.333333,4,0.761266,1.349,OII id by pipeline could be Ha (3rd fit). Ugly...,3,Bad calibration. emission features are secure
393,35191232061767736,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.349,AMA,...,241.123424,0.3337,GALAXY,1.333333,4,0.761266,1.349,OII id by pipeline could be Ha (3rd fit). Ugly...,3,Bad calibration. emission features are secure
443,35191232061767736,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.349,CHH,...,241.123424,0.3337,GALAXY,1.333333,4,0.761266,1.349,OII id by pipeline could be Ha (3rd fit). Ugly...,3,Bad calibration. emission features are secure


conflict ID =  31


In [496]:
#copy this text to a new cell to display the conflict
conflict_id=32
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
322,35191235874390207,QSO,1.1391,SYB,1,S,NaN,NaN,Odd continuum with kinks at B-R; R-Z overlaps,1.1391,QSO,2.0,3,1.1391,0.0,Odd continuum with kinks at B-R; R-Z overlaps|...,3,none
372,35191235874390207,QSO,1.1391,AMA,4,NaN,NaN,NaN,--,1.1391,QSO,2.0,3,1.1391,0.0,Odd continuum with kinks at B-R; R-Z overlaps|...,3,none
422,35191235874390207,QSO,1.1391,CHH,1,NaN,NaN,NaN,--,1.1391,QSO,2.0,3,1.1391,0.0,Odd continuum with kinks at B-R; R-Z overlaps|...,3,none


In [497]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.5224
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. No secure features'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
322,35191235874390207,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1391,SYB,...,61.512937,1.1391,QSO,2.0,3,0.0,1.1391,Odd continuum with kinks at B-R; R-Z overlaps|...,3,Bad calibration. No secure features
372,35191235874390207,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1391,AMA,...,61.512937,1.1391,QSO,2.0,3,0.0,1.1391,Odd continuum with kinks at B-R; R-Z overlaps|...,3,Bad calibration. No secure features
422,35191235874390207,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1391,CHH,...,61.512937,1.1391,QSO,2.0,3,0.0,1.1391,Odd continuum with kinks at B-R; R-Z overlaps|...,3,Bad calibration. No secure features


conflict ID =  32


In [498]:
#copy this text to a new cell to display the conflict
conflict_id=33
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
323,35191235874390797,GALAXY,0.3919,SYB,3,NaN,NaN,NaN,Ha + NII; kink at B-R overlap,0.3919,GALAXY,3.0,2,0.3919,0.0,Ha + NII; kink at B-R overlap|very weak featur...,3,none
373,35191235874390797,GALAXY,0.3919,AMA,2,NaN,NaN,NaN,very weak features but possible emission,0.3919,GALAXY,3.0,2,0.3919,0.0,Ha + NII; kink at B-R overlap|very weak featur...,3,none
423,35191235874390797,GALAXY,0.3919,CHH,4,NaN,NaN,NaN,--,0.3919,GALAXY,3.0,2,0.3919,0.0,Ha + NII; kink at B-R overlap|very weak featur...,3,none


In [499]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.5224
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. emission and aborption features'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
323,35191235874390797,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3919,SYB,...,549.81226,0.3919,GALAXY,3.0,2,0.0,0.3919,Ha + NII; kink at B-R overlap|very weak featur...,3,Bad calibration. emission and aborption features
373,35191235874390797,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3919,AMA,...,549.81226,0.3919,GALAXY,3.0,2,0.0,0.3919,Ha + NII; kink at B-R overlap|very weak featur...,3,Bad calibration. emission and aborption features
423,35191235874390797,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3919,CHH,...,549.81226,0.3919,GALAXY,3.0,2,0.0,0.3919,Ha + NII; kink at B-R overlap|very weak featur...,3,Bad calibration. emission and aborption features


conflict ID =  33


In [500]:
#copy this text to a new cell to display the conflict
conflict_id=34
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
324,35191235874391652,GALAXY,0.1541,SYB,2,NaN,NaN,NaN,--,0.1541,GALAXY,2.666667,2,0.1541,0.0,--|weak features/continuum|--,3,none
374,35191235874391652,GALAXY,0.1541,AMA,2,NaN,NaN,NaN,weak features/continuum,0.1541,GALAXY,2.666667,2,0.1541,0.0,--|weak features/continuum|--,3,none
424,35191235874391652,GALAXY,0.1541,CHH,4,NaN,NaN,NaN,--,0.1541,GALAXY,2.666667,2,0.1541,0.0,--|weak features/continuum|--,3,none


In [501]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.5224
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. absorption features are secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
324,35191235874391652,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.1541,SYB,...,775.699505,0.1541,GALAXY,2.666667,2,0.0,0.1541,--|weak features/continuum|--,3,Bad calibration. absorption features are secure
374,35191235874391652,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.1541,AMA,...,775.699505,0.1541,GALAXY,2.666667,2,0.0,0.1541,--|weak features/continuum|--,3,Bad calibration. absorption features are secure
424,35191235874391652,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.1541,CHH,...,775.699505,0.1541,GALAXY,2.666667,2,0.0,0.1541,--|weak features/continuum|--,3,Bad calibration. absorption features are secure


conflict ID =  34


In [502]:
#copy this text to a new cell to display the conflict
conflict_id=35
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
331,35191235874393030,QSO,1.1442,SYB,1,NaN,NaN,NaN,Odd continuum; no features,1.1442,QSO,1.666667,2,1.1442,0.0,Odd continuum; no features|--|--,3,none
381,35191235874393030,QSO,1.1442,AMA,3,NaN,NaN,NaN,--,1.1442,QSO,1.666667,2,1.1442,0.0,Odd continuum; no features|--|--,3,none
431,35191235874393030,QSO,1.1442,CHH,1,NaN,NaN,NaN,--,1.1442,QSO,1.666667,2,1.1442,0.0,Odd continuum; no features|--|--,3,none


In [503]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.5224
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration.No secure features'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
331,35191235874393030,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1442,SYB,...,170.707358,1.1442,QSO,1.666667,2,0.0,1.1442,Odd continuum; no features|--|--,3,Bad calibration.No secure features
381,35191235874393030,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1442,AMA,...,170.707358,1.1442,QSO,1.666667,2,0.0,1.1442,Odd continuum; no features|--|--,3,Bad calibration.No secure features
431,35191235874393030,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1442,CHH,...,170.707358,1.1442,QSO,1.666667,2,0.0,1.1442,Odd continuum; no features|--|--,3,Bad calibration.No secure features


conflict ID =  35


In [504]:
#copy this text to a new cell to display the conflict
conflict_id=36
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
451,35191235878585750,GALAXY,0.4198,SMC,3,R,0.4184,NaN,--,0.4184,GALAXY,2.333333,2,0.419333,0.000987,--|--|--,3,none
501,35191235878585750,GALAXY,0.4198,AMA,3,NaN,NaN,NaN,--,0.4198,GALAXY,2.333333,2,0.419333,0.000987,--|--|--,3,none
551,35191235878585750,GALAXY,0.4198,CHH,1,NaN,NaN,NaN,--,0.4198,GALAXY,2.333333,2,0.419333,0.000987,--|--|--,3,none


In [505]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.5224
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'No features are secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
451,35191235878585750,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4198,SMC,...,20.628405,0.4184,GALAXY,2.333333,2,0.000987,0.419333,--|--|--,3,No features are secure
501,35191235878585750,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4198,AMA,...,20.628405,0.4198,GALAXY,2.333333,2,0.000987,0.419333,--|--|--,3,No features are secure
551,35191235878585750,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4198,CHH,...,20.628405,0.4198,GALAXY,2.333333,2,0.000987,0.419333,--|--|--,3,No features are secure


conflict ID =  36


In [506]:
#copy this text to a new cell to display the conflict
conflict_id=37
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
453,35191235878587725,GALAXY,0.495,SMC,1,NaN,NaN,NaN,--,0.495,GALAXY,1.666667,2,0.495,0.0,--|--|--,3,none
503,35191235878587725,GALAXY,0.495,AMA,3,NaN,NaN,NaN,--,0.495,GALAXY,1.666667,2,0.495,0.0,--|--|--,3,none
553,35191235878587725,GALAXY,0.495,CHH,1,NaN,NaN,NaN,--,0.495,GALAXY,1.666667,2,0.495,0.0,--|--|--,3,none


In [507]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.3060
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. absoprtion features might be secure. Be conservative'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
453,35191235878587725,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.495,SMC,...,207.646669,0.306,GALAXY,1.666667,2,0.0,0.495,--|--|--,3,Bad calibration. absoprtion features might be ...
503,35191235878587725,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.495,AMA,...,207.646669,0.306,GALAXY,1.666667,2,0.0,0.495,--|--|--,3,Bad calibration. absoprtion features might be ...
553,35191235878587725,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.495,CHH,...,207.646669,0.306,GALAXY,1.666667,2,0.0,0.495,--|--|--,3,Bad calibration. absoprtion features might be ...


conflict ID =  37


In [508]:
#copy this text to a new cell to display the conflict
conflict_id=38
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
457,35191235882780025,GALAXY,0.4639,SMC,4,NaN,NaN,NaN,--,0.4639,GALAXY,3.0,2,0.4639,0.0,--|--|few absorption lines,3,none
507,35191235882780025,GALAXY,0.4639,AMA,3,NaN,NaN,NaN,--,0.4639,GALAXY,3.0,2,0.4639,0.0,--|--|few absorption lines,3,none
557,35191235882780025,GALAXY,0.4639,CHH,2,NaN,NaN,NaN,few absorption lines,0.4639,GALAXY,3.0,2,0.4639,0.0,--|--|few absorption lines,3,none


In [509]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2836
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. absorption features are secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
457,35191235882780025,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4639,SMC,...,351.182741,0.2836,GALAXY,3.0,2,0.0,0.4639,--|--|few absorption lines,3,Bad calibration. absorption features are secure
507,35191235882780025,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4639,AMA,...,351.182741,0.2836,GALAXY,3.0,2,0.0,0.4639,--|--|few absorption lines,3,Bad calibration. absorption features are secure
557,35191235882780025,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4639,CHH,...,351.182741,0.2836,GALAXY,3.0,2,0.0,0.4639,--|--|few absorption lines,3,Bad calibration. absorption features are secure


conflict ID =  38


In [510]:
#copy this text to a new cell to display the conflict
#show_spectra=True
conflict_id=39
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
496,35191235886972941,GALAXY,0.6708,SMC,4,R,0.0225,NaN,--,0.0225,GALAXY,2.333333,3,0.6708,0.634034,--|--|--,3,none
546,35191235886972941,GALAXY,0.6708,AMA,2,NaN,NaN,NaN,--,0.6708,GALAXY,2.333333,3,0.6708,0.634034,--|--|--,3,none
596,35191235886972941,GALAXY,0.6708,CHH,1,NaN,NaN,NaN,--,0.6708,GALAXY,2.333333,3,0.6708,0.634034,--|--|--,3,none


In [511]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0225
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Redshift is secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
496,35191235886972941,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.6708,SMC,...,30.597106,0.0225,GALAXY,2.333333,3,0.634034,0.6708,--|--|--,3,Bad calibration. Redshift is secure
546,35191235886972941,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.6708,AMA,...,30.597106,0.0225,GALAXY,2.333333,3,0.634034,0.6708,--|--|--,3,Bad calibration. Redshift is secure
596,35191235886972941,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.6708,CHH,...,30.597106,0.0225,GALAXY,2.333333,3,0.634034,0.6708,--|--|--,3,Bad calibration. Redshift is secure


conflict ID =  39


In [512]:
#copy this text to a new cell to display the conflict
conflict_id=40
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
462,35191239682818245,QSO,1.1459,SMC,4,R,0.2729,NaN,--,0.2729,QSO,3.0,3,1.1459,0.685835,--|--|--,3,none
512,35191239682818245,QSO,1.1459,AMA,4,NaN,NaN,NaN,--,1.1459,QSO,3.0,3,1.1459,0.685835,--|--|--,3,none
562,35191239682818245,QSO,1.1459,CHH,1,NaN,NaN,NaN,--,1.1459,QSO,3.0,3,1.1459,0.685835,--|--|--,3,none


In [513]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2729
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Redshift is  secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
462,35191239682818245,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1459,SMC,...,111.428282,0.2729,GALAXY,3.0,3,0.685835,1.1459,--|--|--,3,Bad calibration. Redshift is secure
512,35191239682818245,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1459,AMA,...,111.428282,0.2729,GALAXY,3.0,3,0.685835,1.1459,--|--|--,3,Bad calibration. Redshift is secure
562,35191239682818245,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1459,CHH,...,111.428282,0.2729,GALAXY,3.0,3,0.685835,1.1459,--|--|--,3,Bad calibration. Redshift is secure


conflict ID =  40


In [514]:
#copy this text to a new cell to display the conflict
conflict_id=41
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
464,35191239682818588,GALAXY,1.3537,SMC,4,R,0.136,NaN,--,0.1360,GALAXY,3.0,3,1.3537,1.071919,--|--|--,3,none
514,35191239682818588,GALAXY,1.3537,AMA,4,NaN,NaN,NaN,--,1.3537,GALAXY,3.0,3,1.3537,1.071919,--|--|--,3,none
564,35191239682818588,GALAXY,1.3537,CHH,1,NaN,NaN,NaN,--,1.3537,GALAXY,3.0,3,1.3537,1.071919,--|--|--,3,none


In [515]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1360
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Redshift is secure.'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
464,35191239682818588,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.3537,SMC,...,549.786162,0.136,GALAXY,3.0,3,1.071919,1.3537,--|--|--,3,Bad calibration. Redshift is secure.
514,35191239682818588,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.3537,AMA,...,549.786162,0.136,GALAXY,3.0,3,1.071919,1.3537,--|--|--,3,Bad calibration. Redshift is secure.
564,35191239682818588,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.3537,CHH,...,549.786162,0.136,GALAXY,3.0,3,1.071919,1.3537,--|--|--,3,Bad calibration. Redshift is secure.


conflict ID =  41


In [516]:
#copy this text to a new cell to display the conflict
conflict_id=42
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
465,35191239682819126,GALAXY,1.2275,SMC,4,R,0.0818,NaN,--,0.0818,GALAXY,2.333333,3,1.2275,1.059068,--|possible QSO object|--,3,none
515,35191239682819126,GALAXY,1.2275,AMA,2,C,NaN,QSO,possible QSO object,1.2275,QSO,2.333333,3,1.2275,1.059068,--|possible QSO object|--,3,none
565,35191239682819126,GALAXY,1.2275,CHH,1,NaN,NaN,NaN,--,1.2275,GALAXY,2.333333,3,1.2275,1.059068,--|possible QSO object|--,3,none


In [517]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.0818
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Reshift is secure'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
465,35191239682819126,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.2275,SMC,...,5.186262,0.0818,GALAXY,2.333333,3,1.059068,1.2275,--|possible QSO object|--,3,Bad calibration. Reshift is secure
515,35191239682819126,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.2275,AMA,...,5.186262,0.0818,QSO,2.333333,3,1.059068,1.2275,--|possible QSO object|--,3,Bad calibration. Reshift is secure
565,35191239682819126,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.2275,CHH,...,5.186262,0.0818,GALAXY,2.333333,3,1.059068,1.2275,--|possible QSO object|--,3,Bad calibration. Reshift is secure


conflict ID =  42


In [518]:
#copy this text to a new cell to display the conflict
conflict_id=43
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
466,35191239682819256,QSO,1.1431,SMC,2,R,0.6658,NaN,--,0.6658,QSO,2.333333,3,1.1431,0.286529,--|--|--,3,none
516,35191239682819256,QSO,1.1431,AMA,4,NaN,NaN,NaN,--,1.1431,QSO,2.333333,3,1.1431,0.286529,--|--|--,3,none
566,35191239682819256,QSO,1.1431,CHH,1,NaN,NaN,NaN,--,1.1431,QSO,2.333333,3,1.1431,0.286529,--|--|--,3,none


In [519]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.2170
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Secure absorption'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
466,35191239682819256,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1431,SMC,...,55.100799,0.217,GALAXY,2.333333,3,0.286529,1.1431,--|--|--,3,Bad calibration. Secure absorption
516,35191239682819256,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1431,AMA,...,55.100799,0.217,GALAXY,2.333333,3,0.286529,1.1431,--|--|--,3,Bad calibration. Secure absorption
566,35191239682819256,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1431,CHH,...,55.100799,0.217,GALAXY,2.333333,3,0.286529,1.1431,--|--|--,3,Bad calibration. Secure absorption


conflict ID =  43


In [520]:
#copy this text to a new cell to display the conflict
conflict_id=44
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
467,35191239682819376,QSO,1.1362,SMC,2,R,0.1763,NaN,--,0.1763,QSO,2.333333,3,1.1362,0.816033,--|--|--,3,none
517,35191239682819376,QSO,1.1362,AMA,4,NaN,NaN,NaN,--,1.1362,QSO,2.333333,3,1.1362,0.816033,--|--|--,3,none
567,35191239682819376,QSO,1.1362,CHH,1,NaN,NaN,NaN,--,1.1362,QSO,2.333333,3,1.1362,0.816033,--|--|--,3,none


In [521]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.4270
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY' 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration. Possible absorption. Be conservative'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
467,35191239682819376,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1362,SMC,...,120.206741,0.427,GALAXY,2.333333,3,0.816033,1.1362,--|--|--,3,Bad calibration. Possible absorption. Be conse...
517,35191239682819376,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1362,AMA,...,120.206741,0.427,GALAXY,2.333333,3,0.816033,1.1362,--|--|--,3,Bad calibration. Possible absorption. Be conse...
567,35191239682819376,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1362,CHH,...,120.206741,0.427,GALAXY,2.333333,3,0.816033,1.1362,--|--|--,3,Bad calibration. Possible absorption. Be conse...


conflict ID =  44


In [522]:
#copy this text to a new cell to display the conflict
conflict_id=45
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
468,35191239682819437,GALAXY,0.4193,SMC,4,R,0.083,NaN,--,0.0830,GALAXY,3.333333,2,0.4193,0.310526,--|--|--,3,none
518,35191239682819437,GALAXY,0.4193,AMA,4,NaN,NaN,NaN,--,0.4193,GALAXY,3.333333,2,0.4193,0.310526,--|--|--,3,none
568,35191239682819437,GALAXY,0.4193,CHH,2,R,NaN,NaN,--,0.4193,GALAXY,3.333333,2,0.4193,0.310526,--|--|--,3,none


In [523]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift']= 0.0830
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration.  Secure emission feature'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
468,35191239682819437,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4193,SMC,...,1437.559567,0.083,GALAXY,3.333333,2,0.310526,0.4193,--|--|--,3,Bad calibration. Secure emission feature
518,35191239682819437,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4193,AMA,...,1437.559567,0.083,GALAXY,3.333333,2,0.310526,0.4193,--|--|--,3,Bad calibration. Secure emission feature
568,35191239682819437,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4193,CHH,...,1437.559567,0.083,GALAXY,3.333333,2,0.310526,0.4193,--|--|--,3,Bad calibration. Secure emission feature


conflict ID =  45


In [524]:
#copy this text to a new cell to display the conflict
conflict_id=46
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
470,35191239682820184,QSO,1.1227,SMC,3,R,0.6443,NaN,--,0.6443,QSO,2.0,2,1.1227,0.290944,--|weak features and continuum|--,3,none
520,35191239682820184,QSO,1.1227,AMA,2,NaN,NaN,NaN,weak features and continuum,1.1227,QSO,2.0,2,1.1227,0.290944,--|weak features and continuum|--,3,none
570,35191239682820184,QSO,1.1227,CHH,1,NaN,NaN,NaN,--,1.1227,QSO,2.0,2,1.1227,0.290944,--|weak features and continuum|--,3,none


In [525]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift']= 0.6443
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 3
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY' 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration.  Secure emission feature'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
470,35191239682820184,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1227,SMC,...,343.431238,0.6443,GALAXY,2.0,2,0.290944,1.1227,--|weak features and continuum|--,3,Bad calibration. Secure emission feature
520,35191239682820184,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1227,AMA,...,343.431238,0.6443,GALAXY,2.0,2,0.290944,1.1227,--|weak features and continuum|--,3,Bad calibration. Secure emission feature
570,35191239682820184,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1227,CHH,...,343.431238,0.6443,GALAXY,2.0,2,0.290944,1.1227,--|weak features and continuum|--,3,Bad calibration. Secure emission feature


conflict ID =  46


In [526]:
#copy this text to a new cell to display the conflict
conflict_id=47
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
471,35191239682821246,GALAXY,1.0643,SMC,4,R,0.172,NaN,--,0.1720,GALAXY,3.0,3,1.0643,0.761348,--|--|--,3,none
521,35191239682821246,GALAXY,1.0643,AMA,4,NaN,NaN,NaN,--,1.0643,GALAXY,3.0,3,1.0643,0.761348,--|--|--,3,none
571,35191239682821246,GALAXY,1.0643,CHH,1,NaN,NaN,NaN,--,1.0643,GALAXY,3.0,3,1.0643,0.761348,--|--|--,3,none


In [527]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift']= 0.1720
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration.  Secure emission feature'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
471,35191239682821246,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.0643,SMC,...,460.244634,0.172,GALAXY,3.0,3,0.761348,1.0643,--|--|--,3,Bad calibration. Secure emission feature
521,35191239682821246,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.0643,AMA,...,460.244634,0.172,GALAXY,3.0,3,0.761348,1.0643,--|--|--,3,Bad calibration. Secure emission feature
571,35191239682821246,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.0643,CHH,...,460.244634,0.172,GALAXY,3.0,3,0.761348,1.0643,--|--|--,3,Bad calibration. Secure emission feature


conflict ID =  47


In [528]:
len(unique_targets)

54

In [529]:
#copy this text to a new cell to display the conflict
conflict_id=48
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
472,35191239682821445,GALAXY,0.7811,SMC,4,R,0.0817,NaN,--,0.0817,GALAXY,2.666667,3,0.7811,0.646575,--|--|--,3,none
522,35191239682821445,GALAXY,0.7811,AMA,3,NaN,NaN,NaN,--,0.7811,GALAXY,2.666667,3,0.7811,0.646575,--|--|--,3,none
572,35191239682821445,GALAXY,0.7811,CHH,1,NaN,NaN,NaN,--,0.7811,GALAXY,2.666667,3,0.7811,0.646575,--|--|--,3,none


In [530]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift']= 0.0817
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration.  No strong secure feature'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
472,35191239682821445,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.7811,SMC,...,125.276756,0.0817,GALAXY,2.666667,3,0.646575,0.7811,--|--|--,3,Bad calibration. No strong secure feature
522,35191239682821445,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.7811,AMA,...,125.276756,0.0817,GALAXY,2.666667,3,0.646575,0.7811,--|--|--,3,Bad calibration. No strong secure feature
572,35191239682821445,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.7811,CHH,...,125.276756,0.0817,GALAXY,2.666667,3,0.646575,0.7811,--|--|--,3,Bad calibration. No strong secure feature


conflict ID =  48


In [531]:
#copy this text to a new cell to display the conflict
conflict_id=49
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
474,35191239687012856,GALAXY,0.4954,SMC,4,R,0.0923,NaN,--,0.0923,GALAXY,2.333333,3,0.4954,0.369038,--|weak features and continuum|--,3,none
524,35191239687012856,GALAXY,0.4954,AMA,2,NaN,NaN,NaN,weak features and continuum,0.4954,GALAXY,2.333333,3,0.4954,0.369038,--|weak features and continuum|--,3,none
574,35191239687012856,GALAXY,0.4954,CHH,1,NaN,NaN,NaN,--,0.4954,GALAXY,2.333333,3,0.4954,0.369038,--|weak features and continuum|--,3,none


In [532]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift']= 0.0923
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Bad calibration.  Secure emission feature'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
474,35191239687012856,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4954,SMC,...,181.557517,0.0923,GALAXY,2.333333,3,0.369038,0.4954,--|weak features and continuum|--,3,Bad calibration. Secure emission feature
524,35191239687012856,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4954,AMA,...,181.557517,0.0923,GALAXY,2.333333,3,0.369038,0.4954,--|weak features and continuum|--,3,Bad calibration. Secure emission feature
574,35191239687012856,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4954,CHH,...,181.557517,0.0923,GALAXY,2.333333,3,0.369038,0.4954,--|weak features and continuum|--,3,Bad calibration. Secure emission feature


conflict ID =  49


In [533]:
#copy this text to a new cell to display the conflict
conflict_id=50
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
475,35191239687013396,GALAXY,1.4296,SMC,4,R,0.3797,NaN,--,0.3797,GALAXY,2.333333,3,1.4296,0.760963,--|possible QSO|--,3,none
525,35191239687013396,GALAXY,1.4296,AMA,2,RC,1.0837,NaN,possible QSO,1.0837,GALAXY,2.333333,3,1.4296,0.760963,--|possible QSO|--,3,none
575,35191239687013396,GALAXY,1.4296,CHH,1,NaN,NaN,NaN,--,1.4296,GALAXY,2.333333,3,1.4296,0.760963,--|possible QSO|--,3,none


In [534]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift']= 0.3797
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] ='Secure emission feature'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
475,35191239687013396,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.4296,SMC,...,8.704371,0.3797,GALAXY,2.333333,3,0.760963,1.4296,--|possible QSO|--,3,Secure emission feature
525,35191239687013396,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.4296,AMA,...,8.704371,0.3797,GALAXY,2.333333,3,0.760963,1.4296,--|possible QSO|--,3,Secure emission feature
575,35191239687013396,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.4296,CHH,...,8.704371,0.3797,GALAXY,2.333333,3,0.760963,1.4296,--|possible QSO|--,3,Secure emission feature


conflict ID =  50


In [535]:
#copy this text to a new cell to display the conflict
conflict_id=51
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
476,35191239687014075,QSO,1.1359,SMC,4,R,0.1577,NaN,--,0.1577,QSO,3.0,3,1.1359,0.844951,--|--|--,3,none
526,35191239687014075,QSO,1.1359,AMA,4,R,1.0771,NaN,--,1.0771,QSO,3.0,3,1.1359,0.844951,--|--|--,3,none
576,35191239687014075,QSO,1.1359,CHH,1,NaN,NaN,NaN,--,1.1359,QSO,3.0,3,1.1359,0.844951,--|--|--,3,none


In [536]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift']= 0.1577
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] ='Secure emission features'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
476,35191239687014075,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1359,SMC,...,225.586408,0.1577,GALAXY,3.0,3,0.844951,1.1359,--|--|--,3,Secure emission features
526,35191239687014075,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1359,AMA,...,225.586408,0.1577,GALAXY,3.0,3,0.844951,1.1359,--|--|--,3,Secure emission features
576,35191239687014075,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1359,CHH,...,225.586408,0.1577,GALAXY,3.0,3,0.844951,1.1359,--|--|--,3,Secure emission features


conflict ID =  51


In [537]:
#copy this text to a new cell to display the conflict
conflict_id=52
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
481,35191239691206731,GALAXY,0.4953,SMC,4,R,0.5993,NaN,--,0.5993,GALAXY,2.666667,3,0.4953,0.069551,--|--|--,3,none
531,35191239691206731,GALAXY,0.4953,AMA,3,NaN,NaN,NaN,--,0.4953,GALAXY,2.666667,3,0.4953,0.069551,--|--|--,3,none
581,35191239691206731,GALAXY,0.4953,CHH,1,NaN,NaN,NaN,--,0.4953,GALAXY,2.666667,3,0.4953,0.069551,--|--|--,3,none


In [538]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift']= 0.5993
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] ='Secure  feature'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
481,35191239691206731,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4953,SMC,...,47.007968,0.5993,GALAXY,2.666667,3,0.069551,0.4953,--|--|--,3,Secure feature
531,35191239691206731,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4953,AMA,...,47.007968,0.5993,GALAXY,2.666667,3,0.069551,0.4953,--|--|--,3,Secure feature
581,35191239691206731,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.4953,CHH,...,47.007968,0.5993,GALAXY,2.666667,3,0.069551,0.4953,--|--|--,3,Secure feature


conflict ID =  52


In [539]:
#copy this text to a new cell to display the conflict
conflict_id=53
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
485,35191239691208049,QSO,1.1364,SMC,4,R,0.3982,NaN,--,0.3982,QSO,2.666667,3,1.1364,0.527965,--|--|--,3,none
535,35191239691208049,QSO,1.1364,AMA,3,NaN,NaN,NaN,--,1.1364,QSO,2.666667,3,1.1364,0.527965,--|--|--,3,none
585,35191239691208049,QSO,1.1364,CHH,1,NaN,NaN,NaN,--,1.1364,QSO,2.666667,3,1.1364,0.527965,--|--|--,3,none


In [540]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift']= 0.3982
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] ='Secure absorption features'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
485,35191239691208049,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1364,SMC,...,228.980838,0.3982,GALAXY,2.666667,3,0.527965,1.1364,--|--|--,3,Secure absorption features
535,35191239691208049,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1364,AMA,...,228.980838,0.3982,GALAXY,2.666667,3,0.527965,1.1364,--|--|--,3,Secure absorption features
585,35191239691208049,52932,20200303,70500,0.34.4,-1,-1,QSO,1.1364,CHH,...,228.980838,0.3982,GALAXY,2.666667,3,0.527965,1.1364,--|--|--,3,Secure absorption features


conflict ID =  53


### and so on...

We should now recompute the conflicts, and not find any.

In [541]:
vi_conflict = VI.find_conflicts(vi_gp)

In [542]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35191228173649991 35191228211396983 35191239682819126]
Total number of conflicts to resolve:  3


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

In [543]:
#copy this text to a new cell to display the conflict
show_spectra = False
conflict_id=0
display_conflict(conflict_id, show_spectra=show_spectra)


,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
67,35191228173649991,GALAXY,0.1841,AMA,4,C,1.0845,QSO,possible QSO object,1.0845,QSO,1.0,4,0.1841,0.760409,possible QSO object|--,2,disagreement in flag. Ha SII look good
116,35191228173649991,GALAXY,0.1841,CHH,4,NaN,NaN,NaN,--,0.1841,GALAXY,1.0,4,0.1841,0.760409,possible QSO object|--,2,disagreement in flag. Ha SII look good


In [544]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.1841
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'VI_quality'] = 4
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = 'Secure features. Bad calibration. Galaxy'

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
67,35191228173649991,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.1841,AMA,...,2605.849124,0.1841,GALAXY,1.0,4,0.760409,0.1841,possible QSO object|--,2,disagreement in flag. Ha SII look good
116,35191228173649991,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.1841,CHH,...,2605.849124,0.1841,GALAXY,1.0,4,0.760409,0.1841,possible QSO object|--,2,disagreement in flag. Ha SII look good


conflict ID =  0


In [545]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
184,35191228211396983,GALAXY,0.3755,AMA,4,RC,1.0775,NaN,possible QSO,1.0775,GALAXY,2.333333,5,0.3755,0.51036,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,Bad calibration.
234,35191228211396983,GALAXY,0.3755,CHH,4,NaN,NaN,NaN,--,0.3755,GALAXY,2.333333,5,0.3755,0.51036,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,Bad calibration.
284,35191228211396983,GALAXY,0.3755,SYB,4,NaN,NaN,NaN,Ha + abs lines; B-R-Z splice issues?,0.3755,GALAXY,2.333333,5,0.3755,0.51036,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,Bad calibration.


In [546]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.3755
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = '4000 break '

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
184,35191228211396983,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3755,AMA,...,2927.864409,0.3755,GALAXY,2.333333,5,0.51036,0.3755,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,Bad calibration.
234,35191228211396983,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3755,CHH,...,2927.864409,0.3755,GALAXY,2.333333,5,0.51036,0.3755,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,Bad calibration.
284,35191228211396983,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3755,SYB,...,2927.864409,0.3755,GALAXY,2.333333,5,0.51036,0.3755,possible QSO|--|Ha + abs lines; B-R-Z splice i...,3,Bad calibration.


conflict ID =  1


In [547]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=show_spectra)



,TARGETID,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
465,35191239682819126,GALAXY,1.2275,SMC,4,R,0.0818,NaN,--,0.0818,GALAXY,2.333333,3,1.2275,1.059068,--|possible QSO object|--,3,Bad calibration. Reshift is secure
515,35191239682819126,GALAXY,1.2275,AMA,4,C,NaN,QSO,possible QSO object,0.0818,QSO,2.333333,3,1.2275,1.059068,--|possible QSO object|--,3,Bad calibration. Reshift is secure
565,35191239682819126,GALAXY,1.2275,CHH,4,NaN,NaN,NaN,--,0.0818,GALAXY,2.333333,3,1.2275,1.059068,--|possible QSO object|--,3,Bad calibration. Reshift is secure


In [548]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best redshift'] = 0.3755
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TARGETID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
#vi.loc[vi.TARGETID==unique_targets[conflict_id], 'merger comment'] = '4000 break '

#look at the values again to make sure all is well
display(vi[vi.TARGETID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
465,35191239682819126,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.2275,SMC,...,5.186262,0.0818,GALAXY,2.333333,3,1.059068,1.2275,--|possible QSO object|--,3,Bad calibration. Reshift is secure
515,35191239682819126,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.2275,AMA,...,5.186262,0.0818,GALAXY,2.333333,3,1.059068,1.2275,--|possible QSO object|--,3,Bad calibration. Reshift is secure
565,35191239682819126,52932,20200303,70500,0.34.4,-1,-1,GALAXY,1.2275,CHH,...,5.186262,0.0818,GALAXY,2.333333,3,1.059068,1.2275,--|possible QSO object|--,3,Bad calibration. Reshift is secure


conflict ID =  2


Let's check the number of conflicts again.

In [549]:
vi_conflict = VI.find_conflicts(vi_gp)

In [550]:
unique_targets = np.unique(vi_conflict['TARGETID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [551]:
vi['vi_combined_flag'] = vi.groupby('TARGETID')['VI_quality'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TARGETID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [552]:
vi.keys()

Index(['TARGETID', 'EXPID', 'NIGHT', 'TILEID', 'Spec_version',
       'Redrock_version', 'Template_version', 'Redrock_spectype', 'Redrock_z',
       'VI_scanner', 'VI_quality', 'VI_issue', 'VI_z', 'VI_spectype',
       'VI_comment', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G',
       'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2', 'best redshift',
       'best spectype', 'vi_combined_flag', 'vi_diff', 'dz', 'vi_combined_z',
       'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [553]:
print(VI_dir+save_filename)

/global/homes/t/tlan/projects/VI_files/Andes_reinspection/BGS/truth_table_Andes_reinspection_BGS_70500_20200303_3


In [554]:
vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',  
      'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER'].first().to_csv(VI_dir+save_filename)

<ipython-input-554-2063ed20beca>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  vi_gp['Redrock_z', 'Redrock_spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',


Check that merged file reads in OK - check comments

In [555]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [556]:
merged_file.keys()

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'all VI comments',
       'merger comment', 'N_VI', 'DELTACHI2', 'FIBER'],
      dtype='object')

In [558]:
len(merged_file)

200

In [ ]:
VI_dir